In [ ]:
! pip install import-ipynb

In [2]:
from model import BodyposeModel
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel

/tmp/ipykernel_24995/2130244665.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [3]:
NO_OF_OUTPUTS = 21

In [4]:
backbone_model_path = '../model/body_pose_model.pth'

In [5]:
backbone_model = BodyposeBackbone()

In [6]:
backbone_params = torch.load(backbone_model_path)

In [7]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [8]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [9]:
backbone_model.eval()

BodyposeBackbone(
  (model): Sequential(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv1_1): ReLU(inplace=True)
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv1_2): ReLU(inplace=True)
    (pool1_stage1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv2_1): ReLU(inplace=True)
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv2_2): ReLU(inplace=True)
    (pool2_stage1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv3_1): ReLU(inplace=True)
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv3_2): ReLU(inplace=True)
    (conv3_3): Conv2d(256, 256, kernel_size

In [10]:
main_model = MainModel(NO_OF_OUTPUTS)

In [11]:
def train(backbone_model, main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()
    main_model.train()

    out1 = backbone_model(inputs)
    logits = main_model(out1)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    return output.item()

In [12]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [13]:
epochs = 10
batch_size = 4
num_of_batches = Xtrain_torch.size[0] // batch_size

for epoch in range(epochs):
    for batch in range(num_of_batches):
        inputs = Xtrain_torch[batch*batch_size:(batch+1)*batch_size]
        outputs = Ytrain_torch[batch*batch_size:(batch+1)*batch_size]
        output = train(backbone_model, main_model, loss, optimizer, inputs, outputs)

NameError: name 'Xtrain_torch' is not defined